In [1]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 5.4 MB/s 


In [3]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [5]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [6]:
!ls

 a0529.csv   a0607.csv		    Github
 a0530.csv   a0608.csv		    heatmap530.ipynb
 a0531.csv   a0609.csv		    heatmap531.ipynb
 a0601.csv   a0610.csv		    Heat_Map_Online_language.ipynb
 a0602.csv   a0611.csv		    __MACOSX
 a0603.csv   csv		    non_violant1.csv
 a0604.csv   csv.zip		    non_violant2.csv
 a0605.csv   dataframe_csv	    violant.csv
 a0606.csv   dataframe_result_csv  '新建 Microsoft PowerPoint 演示文稿.pptx'


In [7]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [8]:
dataframe_path_530 = 'dataframe_csv/dataframe0605.csv'

In [9]:
dataframe_530 = pd.read_csv(dataframe_path_530)

In [10]:
dataframe_530=pd.DataFrame(dataframe_530,columns=['0','1'])

In [11]:
dataframe_530

,0,1
0,minneapolis,4133
1,america,1268
2,u.s.,815
3,minnesota,750
4,texas,632
...,...,...
2624,auburndale,1
2625,dundee,1
2626,mulberry,1
2627,fort meade,1


In [12]:
dataframe_530.shape[1]

2

In [13]:
columns2=['City','Number']

In [14]:
#len(columns1)
len(columns2)

2

In [15]:
dataframe_530.columns=columns2

In [16]:
dataframe_530

,City,Number
0,minneapolis,4133
1,america,1268
2,u.s.,815
3,minnesota,750
4,texas,632
...,...,...
2624,auburndale,1
2625,dundee,1
2626,mulberry,1
2627,fort meade,1


In [17]:
list_city=dataframe_530['City'].tolist()
list_num=dataframe_530['Number'].tolist()
list_lat=[]
list_lon=[]
list_address=[]

In [18]:
len(list_city)

2629

In [19]:
del_list=[]

In [20]:
from geopy.exc import GeocoderTimedOut
counter=0
for i in list_city:
  #search_string=dataframe_529['City'][i]
  geolocator = Nominatim(user_agent="address_development")
  try:
    location = geolocator.geocode(i,timeout=30)
    if(location==None):
      del_list.append(counter)
      time.sleep(1)
    else:
      list_lat.append(location.latitude)
      list_lon.append(location.longitude)
      list_address.append(location.address)
      time.sleep(1)
    counter+=1
  except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(Location, e.msg))

In [21]:
len(del_list)

258

In [22]:
len(dataframe_530)

2629

In [23]:
for i in del_list:
  dataframe_530=dataframe_530.drop(labels=i,axis=0)

In [24]:
len(dataframe_530)

2371

In [25]:
dataframe_530.reset_index(drop=True, inplace=True)

In [26]:
list_city=dataframe_530['City'].tolist()
list_num=dataframe_530['Number'].tolist()

In [27]:
print(counter)

2629


In [28]:
dict={"lat" : list_lat,
   "lon" : list_lon,
   "address" : list_address}
#dataframe_a1=DataFrame(dict)
#dataframe_a1

In [29]:
from pandas.core.frame import DataFrame
dataframe_a1=DataFrame(dict)
dataframe_a1

,lat,lon,address
0,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
2,48.684067,9.009582,"Unterer See, Bahnhof, Böblingen (Stadt), Böbli..."
3,45.989659,-94.611329,"Minnesota, United States"
4,31.816038,-99.512099,"Texas, United States"
...,...,...,...
2366,28.065297,-81.788691,"Auburndale, Polk County, Florida, 33823, Unite..."
2367,56.460594,-2.970190,"Dundee, Dundee City, Scotland, DD1 1SA, United..."
2368,37.556991,-94.621903,"Mulberry, Crawford County, Kansas, United States"
2369,27.752253,-81.801749,"Fort Meade, Polk County, Florida, United States"


In [30]:
dataframe_530

,City,Number
0,minneapolis,4133
1,america,1268
2,u.s.,815
3,minnesota,750
4,texas,632
...,...,...
2366,auburndale,1
2367,dundee,1
2368,mulberry,1
2369,fort meade,1


In [31]:
len(list_lat)

2371

In [32]:
dataframe_result=pd.concat([dataframe_530,dataframe_a1],axis=1)
dataframe_result

,City,Number,lat,lon,address
0,minneapolis,4133,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,america,1268,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
2,u.s.,815,48.684067,9.009582,"Unterer See, Bahnhof, Böblingen (Stadt), Böbli..."
3,minnesota,750,45.989659,-94.611329,"Minnesota, United States"
4,texas,632,31.816038,-99.512099,"Texas, United States"
...,...,...,...,...,...
2366,auburndale,1,28.065297,-81.788691,"Auburndale, Polk County, Florida, 33823, Unite..."
2367,dundee,1,56.460594,-2.970190,"Dundee, Dundee City, Scotland, DD1 1SA, United..."
2368,mulberry,1,37.556991,-94.621903,"Mulberry, Crawford County, Kansas, United States"
2369,fort meade,1,27.752253,-81.801749,"Fort Meade, Polk County, Florida, United States"


In [33]:
pd.set_option('display.max_rows', None)

In [34]:
list_citya=dataframe_result['City'].tolist()
list_numa=dataframe_result['Number'].tolist()
list_lata=dataframe_result['lat'].tolist()
list_lona=dataframe_result['lon'].tolist()
list_addressa=dataframe_result['address'].tolist()

In [35]:
for i in range(len(list_citya)):
  j=i+1
  while(j<len(list_citya)):
    if(list_lata[i]==list_lata[j]):
      if(list_lona[i]==list_lona[j]):
        list_numa[i]+=list_numa[j]
        del list_citya[j]
        del list_numa[j]
        del list_lata[j]
        del list_lona[j]
        del list_addressa[j]
        continue
    j+=1


In [ ]:
dataframe_result

,City,Number,lat,lon,address
0,minneapolis,7746,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,america,1515,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
2,minnesota,1317,45.989659,-94.611329,"Minnesota, United States"
3,u.s.,959,48.684067,9.009582,"Unterer See, Bahnhof, Böblingen (Stadt), Böbli..."
4,us,703,39.783730,-100.445882,United States
5,the united states,678,29.633008,-97.988594,"The Summit, Guadalupe County, Texas, United St..."
6,washington,550,38.895037,-77.036543,"Washington, District of Columbia, United States"
7,houston,478,29.758938,-95.367697,"Houston, Harris County, Texas, United States"
8,los angeles,396,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
9,new york,353,40.712728,-74.006015,"New York, United States"


In [36]:
dict_result_a={"City" : list_citya,
   "Number" : list_numa,
   "lat" : list_lata,
   "lon" : list_lona,
   "address" : list_addressa}
from pandas.core.frame import DataFrame
dataframe_result_a1=DataFrame(dict_result_a)
dataframe_result_a1

,City,Number,lat,lon,address
0,minneapolis,4135,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,america,1268,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
2,u.s.,826,48.684067,9.009582,"Unterer See, Bahnhof, Böblingen (Stadt), Böbli..."
3,minnesota,891,45.989659,-94.611329,"Minnesota, United States"
4,texas,685,31.816038,-99.512099,"Texas, United States"
5,washington,1133,38.895037,-77.036543,"Washington, District of Columbia, United States"
6,the united states,646,29.633008,-97.988594,"The Summit, Guadalupe County, Texas, United St..."
7,us,618,39.783730,-100.445882,United States
8,houston,499,29.758938,-95.367697,"Houston, Harris County, Texas, United States"
9,new york,893,40.712728,-74.006015,"New York, United States"


In [37]:
dataframe_result_a1.to_csv('dataframe_result_605.csv')

In [38]:
dataframe_result_a1

,City,Number,lat,lon,address
0,minneapolis,4135,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,america,1268,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
2,u.s.,826,48.684067,9.009582,"Unterer See, Bahnhof, Böblingen (Stadt), Böbli..."
3,minnesota,891,45.989659,-94.611329,"Minnesota, United States"
4,texas,685,31.816038,-99.512099,"Texas, United States"
5,washington,1133,38.895037,-77.036543,"Washington, District of Columbia, United States"
6,the united states,646,29.633008,-97.988594,"The Summit, Guadalupe County, Texas, United St..."
7,us,618,39.783730,-100.445882,United States
8,houston,499,29.758938,-95.367697,"Houston, Harris County, Texas, United States"
9,new york,893,40.712728,-74.006015,"New York, United States"


In [ ]:
geolocator = Nominatim(user_agent="address_development")
location = geolocator.geocode('the united states	')

print(f'Address Match: {location.address}')
print(f'Coordinates: {(location.latitude, location.longitude)}')

Address Match: The Summit, Guadalupe County, Texas, United States
Coordinates: (29.6330078, -97.98859404468138)
